In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [2]:
prototxtPath=os.path.sep.join([r'C:\Users\user\Desktop\New folder','deploy.prototxt'])
weightsPath=os.path.sep.join([r'C:\Users\user\Desktop\New folder','res10_300x300_ssd_iter_140000.caffemodel'])

In [3]:
prototxtPath

'C:\\Users\\user\\Desktop\\New folder\\deploy.prototxt'

In [4]:
weightsPath

'C:\\Users\\user\\Desktop\\New folder\\res10_300x300_ssd_iter_140000.caffemodel'

In [5]:
net=cv2.dnn.readNet(prototxtPath,weightsPath)

In [6]:
model=load_model(r'C:\Users\user\Desktop\New folder\mobilenet_v2.model')

In [7]:
image=cv2.imread(r'C:\Users\user\Desktop\New folder\example_01.png')

In [8]:
image

array([[[186, 183, 145],
        [186, 184, 144],
        [186, 184, 144],
        ...,
        [223, 220, 215],
        [223, 220, 215],
        [223, 220, 215]],

       [[184, 183, 145],
        [184, 183, 145],
        [184, 183, 145],
        ...,
        [223, 220, 215],
        [223, 220, 215],
        [223, 220, 215]],

       [[182, 181, 146],
        [182, 181, 146],
        [183, 182, 144],
        ...,
        [223, 220, 215],
        [223, 220, 215],
        [223, 220, 215]],

       ...,

       [[ 90,  87,  77],
        [ 90,  87,  77],
        [ 91,  88,  78],
        ...,
        [106, 102,  96],
        [106, 102,  96],
        [106, 102,  96]],

       [[ 90,  87,  77],
        [ 91,  88,  78],
        [ 91,  88,  78],
        ...,
        [105, 101,  95],
        [105, 101,  95],
        [105, 101,  95]],

       [[ 90,  87,  77],
        [ 91,  88,  78],
        [ 91,  88,  78],
        ...,
        [104, 100,  94],
        [104, 100,  94],
        [104, 100,  94]]

In [9]:
(h,w)=image.shape[:2]

In [10]:
(h,w)

(500, 600)

In [11]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [12]:
blob

array([[[[  54.,   44.,   46., ...,  112.,  112.,  112.],
         [  47.,   47.,   50., ...,  112.,  112.,  111.],
         [  31.,   36.,   42., ...,  111.,  111.,  111.],
         ...,
         [ -27.,  -22.,  -21., ...,  118.,   81.,   20.],
         [ -45.,  -31.,  -24., ...,   86.,   20.,  -20.],
         [ -39.,  -29.,  -22., ...,   44.,  -36.,  -31.]],

        [[  11.,    6.,    8., ...,   44.,   44.,   44.],
         [  10.,   10.,   13., ...,   44.,   44.,   43.],
         [  -1.,   -1.,    5., ...,   43.,   43.,   43.],
         ...,
         [ -93.,  -89.,  -88., ...,  -31.,  -59.,  -87.],
         [-108.,  -98.,  -91., ...,  -59.,  -94., -100.],
         [-100.,  -92.,  -84., ...,  -80., -118., -102.]],

        [[  64.,   56.,   58., ...,  101.,  101.,  101.],
         [  58.,   58.,   62., ...,  101.,  101.,  100.],
         [  41.,   46.,   53., ...,  100.,  100.,  100.],
         ...,
         [ -36.,  -32.,  -31., ...,   -2.,  -27.,  -36.],
         [ -51.,  -41.,  -

In [13]:
net.setInput(blob)
detections=net.forward()

In [14]:
detections

array([[[[ 0.00000000e+00,  1.00000000e+00,  6.99217379e-01,
           5.64791501e-01,  1.26157045e-01,  6.98661506e-01,
           3.30779165e-01],
         [ 0.00000000e+00,  1.00000000e+00,  1.47438303e-01,
           7.22443461e-02,  2.58059874e-02,  2.11694688e-01,
           1.78925544e-01],
         [ 0.00000000e+00,  1.00000000e+00,  1.26563773e-01,
           4.15880156e+00,  4.00189447e+00,  4.84034872e+00,
           4.98249865e+00],
         [ 0.00000000e+00,  1.00000000e+00,  1.23299815e-01,
           1.63142830e-01,  3.99527764e+00,  8.52543116e-01,
           4.97916412e+00],
         [ 0.00000000e+00,  1.00000000e+00,  1.21046200e-01,
           2.53547728e-02,  1.25362217e-01,  9.87788200e-01,
           8.76508653e-01],
         [ 0.00000000e+00,  1.00000000e+00,  1.19359128e-01,
           4.01280355e+00,  1.30149603e-01,  4.97421598e+00,
           8.82615566e-01],
         [ 0.00000000e+00,  1.00000000e+00,  1.07346699e-01,
           5.39699852e-01,  6.90373003e

In [ ]:
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    if confidence>0.5:
        #we need the X,Y coordinates
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        #ensure the bounding boxes fall within the dimensions of the frame
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        
        #extract the face ROI, convert it from BGR to RGB channel, resize it to 224,224 and preprocess it
        face=image[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        #determine the class label and color we will use to draw the bounding box and text
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        #include the probability in the label
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        #display the label and bounding boxes
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
        
        
cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()